In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from tensorflow.keras.utils import plot_model
import matplotlib.pyplot as plt
import json

In [2]:
import sys 
sys.path.append("../code/")
from model import SNForecastModel

In [3]:
data = np.load("../data/padded_x_train.npy")
data[:,:,2] -= 1 
num_features = data.shape[2]
X_train, y_train = data[:,:-3,:],  data[:,-3:,:]

In [4]:
data_val = np.load("../data/padded_x_val.npy")
data_val[:,:,2] -= 1 

X_val, y_val = data_val[:,:-3,:],  data_val[:,-3:,:]

In [5]:
model = SNForecastModel(units=32, out_steps=3)
losses = {
    "out_dt": "mse",
    "out_mag": "mse"
}
model.compile(optimizer="rmsprop", loss=losses)

In [6]:
x_dt = np.expand_dims(X_train[:,:,0],axis=2)
x_mag = np.expand_dims(X_train[:,:,1],axis=2)
y_dt = np.expand_dims(y_train[:,:,0],axis=2)
y_mag = np.expand_dims(y_train[:,:,1],axis=2)
#-----------------------VALIDATION-------------------------------
x_val_dt = np.expand_dims(X_val[:,:,0],axis=2)
x_val_mag = np.expand_dims(X_val[:,:,1],axis=2)
y_val_dt = np.expand_dims(y_val[:,:,0],axis=2)
y_val_mag = np.expand_dims(y_val[:,:,1],axis=2)

In [7]:
def normalize(inputs, min_val = None, max_val = None):
    if min_val is None:
        min_val = inputs[inputs > 0].min()
    if max_val is None:
        max_val = inputs[inputs > 0].max()
    
    inputs[inputs > 0] = (inputs[inputs > 0] - min_val) /(max_val - min_val)
    return inputs, min_val, max_val

In [8]:
x_dt, dt_min, dt_max = normalize(x_dt)
y_dt, _ ,_ = normalize(y_dt, min_val=dt_min, max_val=dt_max)
x_val_dt, _ , _  = normalize(x_val_dt, min_val=dt_min, max_val=dt_max)
y_val_dt, _ , _  = normalize(y_val_dt, min_val=dt_min, max_val=dt_max)

np.savez("../data/dt_norm", {"dt_min": dt_min, "dt_max":dt_max})

In [9]:
x_mag, mag_min, mag_max = normalize(x_mag)
y_mag,_ , _ = normalize(y_mag, min_val= mag_min, max_val= mag_max)
x_val_mag, _ , _ = normalize(x_val_mag, min_val=mag_min, max_val=mag_max)
y_val_mag, _ , _ = normalize(y_val_mag, min_val=mag_min, max_val=mag_max)

np.savez("../data/mag_norm", {"mag0":{"min":mag_min, "max":mag_max}})

In [10]:
ins = {
    "dt_in":x_dt,
    "mag_in": x_mag,
}
outs = {"out_dt":y_dt,
        "out_mag":y_mag}
#-----------------------VALIDATION-------------------------------
ins_val = {
    "dt_in":x_val_dt,
    "mag_in": x_val_mag,
}
outs_val = {"out_dt":y_val_dt,
            "out_mag": y_val_mag}

In [11]:
#Early stops
early_stop = tf.keras.callbacks.EarlyStopping( monitor='val_loss', min_delta=1e-15, patience=5)

#Tensorboard
tensorboard = tf.keras.callbacks.TensorBoard("../data/training/logs")

#Checkpoint
checkpoint = tf.keras.callbacks.ModelCheckpoint("../data/training/model_checkpoints/checkpoint", monitor='val_loss', verbose=0, save_best_only=True)

callbacks = [tensorboard,checkpoint, early_stop] # mag_early_stop,fid_early_stop,dt_early_stop,

In [13]:
MAX_EPOCHS=100
history = model.fit(ins,outs,
                    batch_size=1000, 
                    epochs=MAX_EPOCHS, 
                    validation_data=(ins_val,outs_val), 
                    callbacks=callbacks)

Epoch 1/100
26/26 [==============================] - 13s 512ms/step - loss: 0.5740 - out_dt_loss: 0.0364 - out_mag_loss: 0.5377 - val_loss: 0.5697 - val_out_dt_loss: 0.0369 - val_out_mag_loss: 0.5328
Epoch 2/100
26/26 [==============================] - 17s 672ms/step - loss: 0.5740 - out_dt_loss: 0.0364 - out_mag_loss: 0.5377 - val_loss: 0.5697 - val_out_dt_loss: 0.0369 - val_out_mag_loss: 0.5328
Epoch 3/100
26/26 [==============================] - 17s 664ms/step - loss: 0.5740 - out_dt_loss: 0.0364 - out_mag_loss: 0.5377 - val_loss: 0.5697 - val_out_dt_loss: 0.0369 - val_out_mag_loss: 0.5328
Epoch 4/100
26/26 [==============================] - 17s 668ms/step - loss: 0.5740 - out_dt_loss: 0.0364 - out_mag_loss: 0.5377 - val_loss: 0.5697 - val_out_dt_loss: 0.0369 - val_out_mag_loss: 0.5328
Epoch 5/100
26/26 [==============================] - 17s 642ms/step - loss: 0.5740 - out_dt_loss: 0.0364 - out_mag_loss: 0.5377 - val_loss: 0.5697 - val_out_dt_loss: 0.0369 - val_out_mag_loss: 0.5328


In [14]:
history_dict = history.history
json.dump(history_dict, open("../data/training/history_model.json", 'w'))

In [15]:
#model.save("../data/models/sn_model.h55")

In [16]:
data_test = np.load("../data/padded_x_val.npy")

In [19]:
sample = data_test[0]

In [22]:
sample = sample[~np.any(sample==-1,axis=1)]

In [47]:
sample

array([[-1.2873765e-05, -1.5244632e+00,  1.0000000e+00],
       [-1.3952655e-05, -1.5295479e+00,  1.0000000e+00],
       [-2.9167159e-05, -1.5335115e+00,  1.0000000e+00],
       [-1.0556985e-05, -1.5299405e+00,  1.0000000e+00],
       [-2.7650613e-05, -1.5201411e+00,  1.0000000e+00],
       [-2.6102955e-05, -1.5121243e+00,  1.0000000e+00]], dtype=float32)

In [36]:
x_sample = np.expand_dims(sample[:-3,:],axis=0)
y_sample = np.expand_dims(sample[-3:,:],axis=0)

In [41]:
x_sample_dt,_,_ = normalize(x_sample[:,:,0], min_val=dt_min, max_val=dt_max)
dt_x_sample = np.expand_dims(x_sample_dt ,axis=2)
y_sample_dt,_,_ = normalize(y_sample[:,:,0], min_val=dt_min, max_val=dt_max)
dt_y_sample = np.expand_dims(y_sample_dt,axis=2)

mag_x_sample,_,_ = normalize(x_sample[:,:,1], min_val=mag_min, max_val=mag_max)
mag_y_sample,_,_ = normalize(y_sample[:,:,1],min_val=mag_min, max_val=mag_max)

In [43]:
mag_y_sample

array([[-1.5299405, -1.5201411, -1.5121243]], dtype=float32)

In [46]:
np.expand_dims(dt_x_sample,axis=2)

array([[[[-1.2873765e-05]],

        [[-1.3952655e-05]],

        [[-2.9167159e-05]]]], dtype=float32)

In [45]:
ins_sample = {
    "dt_in":np.expand_dims(dt_x_sample,axis=2),
    "mag_in": np.expand_dims(mag_x_sample,axis=2),
}
y_hat = model.predict(ins_sample)

ValueError: in user code:

    /home/camilo/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1147 predict_function  *
        outputs = self.distribute_strategy.run(
    ../code/model.py:67 call  *
        prediction_dt, prediction_mag, states = self.warmups(inputs)
    ../code/model.py:43 warmups  *
        inputs = self.concat([dt_in, mag_in])
    /home/camilo/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/base_layer.py:927 __call__  **
        outputs = call_fn(cast_inputs, *args, **kwargs)
    /home/camilo/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/layers/merge.py:183 call
        return self._merge_function(inputs)
    /home/camilo/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/layers/merge.py:522 _merge_function
        return K.concatenate(inputs, axis=self.axis)
    /home/camilo/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/backend.py:2709 concatenate
        return array_ops

In [ ]:
y_hat

In [ ]:
mag_hat = y_hat["out"][0,:,1]
mag_hat = mag_hat * (mag0_max-mag0_min) + mag0_min

In [ ]:
dts = np.hstack([dt_x_sample[0,:,0],dt_y_sample[0,:,0]])
times = np.cumsum(dts) * (dt_max-dt_min) + dt_min
mag = np.hstack([mag_x_sample[0,:,0],mag0_y_sample[0,:]])
mag = mag * (mag0_max-mag0_min) + mag0_min

y_dts = np.hstack([dt_x_sample[0,:,0],y_hat["out"][0,:,0]])
y_times =  np.cumsum(dts) * (dt_max-dt_min) + dt_min
y_times = y_times[-3:]
y_mag = mag_hat * (mag0_max-mag0_min) + mag0_min

plt.scatter(times, mag)

plt.scatter(y_times, y_mag, label="Pred_1")

